# “Without pain, without sacrifice, we would have nothing.”  -- Fight Club

## Recap

### Main

In [1]:
%%writefile main.kv

<Widget>:
    canvas:
        Color:
            rgba: .5,.5,.5,.5
        Line:
            rectangle: self.x, self.y, self.width, self.height
            width: 2

Overwriting main.kv


In [2]:
%%writefile main.py

from kivy.app import App
from selectiontool import SelectionTool


class MainApp(App):
    title = 'Image Selection Tool'

    def build(self):
        return SelectionTool()


if __name__ == '__main__':
    MainApp().run()

Overwriting main.py


### ImagePane

In [3]:
%%writefile imagepane.kv

<ImagePane>:
    allow_stretch: True
    on_size: self.redraw_rectangles()
    on_pos: self.redraw_rectangles()


Overwriting imagepane.kv


In [4]:
%%writefile imagepane.py

from kivy.uix.image import Image
from kivy.lang import Builder
from kivy.app import App

from selectionbox import SelectionBox

Builder.load_file("imagepane.kv")


class ImagePane(Image):

    drawing_rectangle = None
    rectangles = []

    
    def __init__(self, **kwargs):
        super(ImagePane, self).__init__(**kwargs)
        self.register_event_type('on_store_rectangles')
    
    def on_store_rectangles(self, *args, **kwargs):
        pass

    def on_touch_move(self, touch):
        if self.collide_point(*touch.pos):
            pos = [min(touch.pos[n], touch.opos[n]) for n in [0, 1]]
            size = [abs(touch.pos[n] - touch.opos[n]) for n in [0, 1]]
            if self.drawing_rectangle is None:
                self.drawing_rectangle = SelectionBox(pos=pos, size=size, image_pane=self)
                self.add_new_rectangle(self.drawing_rectangle)
            else:
                self.drawing_rectangle.pos = pos
                self.drawing_rectangle.size = size

    def on_touch_up(self, touch):
        if self.drawing_rectangle:
            self.drawing_rectangle.compute_unit_coordinates()
            self.drawing_rectangle = None
            self.store_rectangles()

    def add_new_rectangle(self, rect):
        self.add_widget(rect)
        self.rectangles.append(rect)
        
    def delete_last_rectangle(self):
        if self.rectangles:
            bad_rectangle = self.rectangles.pop()
            self.remove_widget(bad_rectangle)
            self.store_rectangles()
            
    def clear_rectangles(self):
        self.rectangles = []
        self.clear_widgets()

    def store_rectangles(self):
        self.dispatch('on_store_rectangles', rectangles=self.rectangles)

    def redraw_rectangles(self):
        for rect in self.rectangles:
            rect.compute_screen_coordinates()


Overwriting imagepane.py


### SelectionBox

In [5]:
%%writefile selectionbox.kv

<SelectionBox>:
    label: _label
    color: (1, 0, 0, 1)
    image_pane: None
    canvas:
        Color:
            rgba: root.color
        Line:
            width: 2.
            rectangle: (self.x, self.y, self.width, self.height)
    TextInput:
        id: _label
        multiline: False
        size_hint: 1, None
        height: 50
        center: root.center
        on_text: root.image_pane.store_rectangles() if root.image_pane else None


Overwriting selectionbox.kv


In [6]:
%%writefile selectionbox.py

from kivy.uix.widget import Widget
from kivy.lang import Builder

Builder.load_file('selectionbox.kv')


class SelectionBox(Widget):

    def __init__(self, image_pane, text='', pos=None, size=None, unit_pos=None, unit_size=None):
        super(SelectionBox, self).__init__(pos=pos, size=size)
        self.label.text = text
        self.image_pane = image_pane
        self.unit_pos = unit_pos
        self.unit_size = unit_size
        
    def to_dict(self):
        return {'text': self.label.text, 'pos': self.pos, 'size': self.size,
                'unit_pos': self.unit_pos, 'unit_size': self.unit_size}

    def compute_unit_coordinates(self):
        image_pos = [self.image_pane.pos[n] + (self.image_pane.size[n] - self.image_pane.norm_image_size[n]) / 2
                     for n in (0, 1)]
        self.unit_pos = tuple([(self.pos[n] - image_pos[n]) * 1.0 / self.image_pane.norm_image_size[n] for n in [0, 1]])
        self.unit_size = tuple([self.size[n] * 1.0 / self.image_pane.norm_image_size[n] for n in [0, 1]])

    def compute_screen_coordinates(self, *_):
        image_pos = [self.image_pane.pos[n] + (self.image_pane.size[n] - self.image_pane.norm_image_size[n]) / 2
                     for n in (0, 1)]
        self.pos = [self.unit_pos[n] * self.image_pane.norm_image_size[n] + image_pos[n] for n in [0, 1]]
        self.size = [self.unit_size[n] * self.image_pane.norm_image_size[n] for n in [0, 1]]


Overwriting selectionbox.py


### SelectionTool

In [7]:
%%writefile selectiontool.kv

<SelectionTool>:
    orientation: "vertical"

    book_selector: _book_selector
    page_selector: _page_selector
    image_pane: _image_pane
    word_list: _word_list

    book_id: self.book_selector.text
    page: self.page_selector.text

    BoxLayout:
        orientation: 'horizontal'

        Label: # Spacer
            size_hint_x: 4

        Spinner:
            id: _book_selector

        Spinner:
            id: _page_selector

    Button:
        text: "Delete Last Rectangle"
        on_press: root.image_pane.delete_last_rectangle()

    BoxLayout:
        orientation: 'horizontal'
        size_hint_y: 16
        BoxLayout:
            id: _word_list
            size_hint_x: 0.2
            orientation: 'vertical'

        ImagePane:
            id: _image_pane
            size_hint_x: 0.8
            source: ''

Overwriting selectiontool.kv


In [2]:
%%writefile selectiontool.py

import os
import glob
import json

from kivy.uix.boxlayout import BoxLayout
from kivy.uix.label import Label
from kivy.properties import StringProperty
from kivy.lang import Builder
from kivy.app import App
import kivy 

from imagepane import ImagePane
from selectionbox import SelectionBox

Builder.load_file('selectiontool.kv')


class SelectionTool(BoxLayout):
    library_directory = 'Example_Data'
    book_id = StringProperty()
    page = StringProperty()

    def __init__(self):
        super(SelectionTool, self).__init__()
        self.image_pane.bind(on_store_rectangles=self.store_rectangles)
        self.all_rectangles = {}
        self.rectangles_filename = 'rectangles.json'
        self.load_rectangles()
        
        book_pattern = os.path.join(self.library_directory, '[0-9]' * 4)
        self.book_selector.values = [os.path.basename(s) for s in glob.glob(book_pattern)]
        self.book_selector.text = self.book_selector.values[0] if self.book_selector.values else 'No Books'

        self.on_book_id()
        
    def on_book_id(self, inst=None, value=None):
        image_pattern = os.path.join(self.library_directory, self.book_id, '*.jpg')
        self.page_selector.values = [os.path.basename(s)[:-4] for s in glob.glob(image_pattern)]
        self.page_selector.text = self.page_selector.values[0] if self.page_selector.values else 'No Images'
        
        self.word_list.clear_widgets()
        with open(os.path.join(self.library_directory, self.book_id, 'word_list.txt')) as fp:
            for word in sorted(fp.readlines()):
                self.word_list.add_widget(Label(text=word.strip()))
        self.color_word_list()
        
        self.on_page()
    
    def on_page(self, *_):
        page_filename = self.page + '.jpg'
        self.image_pane.source = os.path.join(self.library_directory, self.book_id, page_filename)
        self.image_pane.clear_rectangles()
        try:
            for rect in self.all_rectangles[self.book_id][self.page]:
                rect.compute_screen_coordinates()
                self.image_pane.add_new_rectangle(rect)
        except KeyError:
            pass
        
    def store_rectangles(self, sender=None, rectangles=[]):
        if self.book_id not in self.all_rectangles:
            self.all_rectangles[self.book_id] = {}
        self.all_rectangles[self.book_id].update({self.page: [r for r in rectangles]})
        self.color_word_list()
        self.save_rectangles()

    def load_rectangles(self):
        self.all_rectangles = {}
        try:
            with open(self.rectangles_filename) as fd:
                all_rectangles_dict = json.load(fd)
                for book_id, book_rectangles in all_rectangles_dict.items():
                    self.all_rectangles[book_id] = {}
                    for page, rectangles in book_rectangles.items():
                        self.all_rectangles[book_id][page] = \
                            [SelectionBox(image_pane=self.image_pane, **rect) for rect in rectangles]
        except IOError:
            print("Can't find rectangles file!")
        
    def save_rectangles(self):
        rectangle_dict = {}
        for book_id, book_rectangles in self.all_rectangles.items():
            for page, rectangles in book_rectangles.items():
                page_dict = {page: [rect.to_dict() for rect in rectangles]}
                if rectangles:
                    rectangle_dict[book_id] = rectangle_dict.get(book_id, {})
                    rectangle_dict[book_id].update(page_dict)
        
        with open(self.rectangles_filename, 'w') as fd:
            json.dump(rectangle_dict, fd, sort_keys=True, indent=4, separators=(',', ': '))
     
    def color_word_list(self):
        if self.book_id in self.all_rectangles:
            rectangle_labels = [rect.label.text for page_rects in self.all_rectangles[self.book_id].values()
                                for rect in page_rects]
        else:
            rectangle_labels = []
            
        for label in self.word_list.children:
            label.color = (0, 1, 0, 1) if label.text in rectangle_labels else (1, 1, 1, 1)


Overwriting selectiontool.py


# Getting this working on iOS

You'll need to be using a Mac for this, unfortunately.

First we need to modify our code slightly. Apple restricts where we can write data. In particular, we can't write it in the same location as the files we will install on the device.

Instead, iOS provides a documents directory for us. Kivy gives us the location (as well as a similar good location on Android and other platforms) via the `user_data_dir` property.

Note that above we snuck in an `import kivy` into `selectiontool.py`.

In [3]:
%%writefile -a selectiontool.py

    def __init__(self):
        super(SelectionTool, self).__init__()
        self.image_pane.bind(on_store_rectangles=self.store_rectangles)
        self.all_rectangles = {}
        if kivy.platform == 'ios': # <-- deal with iOS
            self.rectangles_filename = os.path.join(App.get_running_app().user_data_dir, 'rectangles.json')
        else:
            self.rectangles_filename = 'rectangles.json'
        self.load_rectangles()
        
        book_pattern = os.path.join(self.library_directory, '[0-9]' * 4)
        self.book_selector.values = [os.path.basename(s) for s in glob.glob(book_pattern)]
        self.book_selector.text = self.book_selector.values[0] if self.book_selector.values else 'No Books'


Appending to selectiontool.py


In [10]:
#!python main.py

## kivy-ios

Install https://github.com/kivy/kivy-ios as described in the Readme.

Build the kivy files.

This will take a long time, maybe half an hour or so.

## Build the project

Assuming you've checked out `kivy-ios` and this repo at the same level, go into the main dir of this repo and:

Boom! You have an xCode project! Well, unless something didn't work.  `¯\_(ツ)_/¯`

Assuming things worked...

## Edit and run the project!

You'll have to edit a few things to get it to work. In particular, 
* Build Settings->Build Options->Enable Bitcode set to "No"
<img src="Images/disable_bitcode.png"/>
* add some libraries that aren't there by default unfortunately: libz.tbd, libsqlite3.tbd, libc++.tbd
<img src="Images/add_libraries.png"/>

At this point you should be able to run on the simulator (note: Sometimes buggy)
<img src="Images/run_on_simulator.png"/>
<img src="Images/running_on_simulator.png"/>

To run on a device, you'll have to set up code signing. 
* To test things on your own device, you don't need to pay, see http://blog.ionic.io/deploying-to-a-device-without-an-apple-developer-account/
* To deploy on other people's devices and put in the App Store, you'll need a paid Apple Developer Account.